In [145]:
import math
import numpy as np
import random
import graphviz

def get_edges_nodes(root):
    nodes = set()
    edges = set()

    def explore(root):
        if root not in nodes:
            nodes.add(root)
            for node in root._prev:
                edges.add((node, root))
                explore(node)

    explore(root)
    return nodes, edges

def draw_plot(root):
    graph = graphviz.Digraph(name='direct graph', graph_attr={'rankdir':'LR'}, node_attr={'shape':'record'})
    nodes, edges = get_edges_nodes(root)
    # draw node
    for node in nodes:
        graph.node(str(id(node)), label='{%s|data %.4f|grad %.4f}' %(node.label, node.data, node.grad))
        if node._op != "":
            graph.node(f'{str(id(node))}_{node._op}', label=node._op, shape='oval')
            graph.edge(f'{str(id(node))}_{node._op}', str(id(node)))
    # draw edge
    for head, tail in edges:
        graph.edge(str(id(head)), f'{str(id(tail))}_{tail._op}')
    return graph

In [254]:
class Value():
    def __init__(self, data, label="", _chidlren=[], _op=""):
        self.data = data
        self.label = label
        self._prev = set(_chidlren)
        self._op = _op
        self.grad = 0
        self._backward = lambda: None

    def __repr__(self):
        return f'Value {self.label} ({self.data})'
        
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(other.data + self.data, _op = "+", _chidlren=[self, other])
        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward
        return out
    
    def __sub__(self, other):
        return self + -1 * other
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(other.data * self.data, _op = "*", _chidlren=[self, other])
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        return out
    
    def tanh(self):
        tanh = math.tanh(self.data)
        out = Value(tanh, _op='tanh', _chidlren=[self])
        def _backward():
            self.grad += (1 - out.data**2) * out.grad
        out._backward = _backward
        return out
    
    def __pow__(self, p):
        out = self.data ** p
        out = Value(out, _op='pow', _chidlren=[self])
        def _backward():
            self.grad += (p * self.data**(p-1)) * out.grad
        out._backward = _backward
        return out
    def __radd__(self, other):
        return self + other
    
    def __rsub__(self, other):
        return self + other
    
    def __rmul__(self, other):
        return self * other
    
    def backward(self):
        self.grad = 1
        def build_topo(self):
            nodes = []
            visited = set()
            def travel(node):
                if node not in visited:
                    for child in node._prev:
                        travel(child)
                    nodes.append(node)
            travel(self)
            return nodes
        nodes = build_topo(self)
        for node in reversed(nodes): node._backward()

In [277]:
class Neuron():
    def __init__(self, n_in):
        self.weights = [Value(random.uniform(-1, 1), label=f'w{i}') for i in range(n_in)]
        self.b = Value(random.uniform(-1, 1), label='b')
        
    def __call__(self, x):
        return sum([w*o for w, o in zip(self.weights, x)], self.b)
    
    def parameters(self):
        return [p for p in [self.b] + self.weights]
    
class Layer():
    def __init__(self, n_in, n_out):
        self.neurons = [Neuron(n_in) for _ in range(n_out)]
    
    def __call__(self, x):
        return [neuron(x) for neuron in self.neurons]
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP():
    def __init__(self, n_int, n_outs):
        sizes = [n_int] + n_outs
        self.layers = [Layer(sizes[i], sizes[i+1]) for i in range(len(n_outs))]
        
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        x = [o.tanh() for o in x]
        return x if len(x) != 1 else x[0]
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

net = MLP(3, [4, 4, 1])
xs = [
    [3.0, 2.0, 1.0],
    [-1.0, 3.0, 1.0],
    [-1.0, -1.0, 1.0],
    [-1.0, -2.0, -1.0],
]
ys = [1, -1, -1, 1]

In [348]:
# predict and loss

y_pred = [net(x) for x in xs]
print(y_pred)
L = sum([(ygt - yout)**2 for ygt, yout in zip(ys, y_pred)])
print(L)
# zero grad 
for p in net.parameters(): p.grad = 0
L.backward()
for p in net.parameters(): p.data += -0.01 * p.grad

[Value  (-0.9744043667061172), Value  (0.9999994330918006), Value  (0.9264094680516899), Value  (-0.9396184549527615)]
Value  (0.009716633818763247)
